In [ ]:
# default_exp wrangler

# wrangler

> The Wrangler object is a helper for easily deploying docker images on AWS EC2 Instances

In [ ]:
# export
from typing import Any, Dict

import boto3


class Wrangler:
    "Handler for deploying a docker image on one or more AWS EC2 Instances"

    def __init__(self):
        # TODO: We won't be able to use mfa.  Need to
        # have this take in access, secret, region
        # once escalated account is obtained for gitlab runner.
        self._ssm_client = boto3.client("ssm")

    def execute_command(self, instance_id: str, command: str) -> Dict[str, Any]:
        response = self._ssm_client.send_command(
            DocumentName="AWS-RunShellScript",
            Parameters={"commands": [command]},
            InstanceIds=[instance_id],
        )
        command_id = response["Command"]["CommandId"]
        instance_id = response["Command"]["InstanceIds"][0]

        return {
            "instance_id": instance_id,
            "command_id": command_id,
        }

    def get_commands(self, instance_id: str = None):
        if instance_id:
            return self._ssm_client.list_commands(InstanceId=instance_id)
        else:
            return self._ssm_client.list_commands()

    def get_command_status(self, instance_id: str, command_id: str):
        response = self._ssm_client.get_command_invocation(
            CommandId=command_id, InstanceId=instance_id,
        )
        return response["Status"]

In [ ]:
#hide

# Mock the boto3.client object

#Why are we mocking this?

#This is a wrapper library.  We don't want to test boto3.  Amazon has that covered.
#We also don't want every commit into the repo to trigger actual AWS code, so the documentation here is going to double as tests.  Mocks will be marked as appropriate.

In [ ]:
#hide
from unittest.mock import Mock

boto3.client = Mock()
boto3.client.return_value = Mock()

In [ ]:
from nbdev.showdoc import *
show_doc(Wrangler.execute_command)

<h4 id="Wrangler.execute_command" class="doc_header"><code>Wrangler.execute_command</code><a href="__main__.py#L16" class="source_link" style="float:right">[source]</a></h4>

> <code>Wrangler.execute_command</code>(**`instance_id`**:`str`, **`command`**:`str`)



In [ ]:
wrangler = Wrangler()

In [ ]:
#hide
# Set AWS boto3 client's return value for testing purposes.
wrangler._ssm_client.send_command.return_value = {"Command": {
            "InstanceIds": ["some_instance_id"],
            "CommandId": "docker_command_id",
    }}

In [ ]:
executed_command = wrangler.execute_command("some_instance_id", "docker")
executed_command

{'instance_id': 'some_instance_id', 'command_id': 'docker_command_id'}

In [ ]:
#hide
assert executed_command == {'instance_id': 'some_instance_id', 'command_id': 'docker_command_id'}

In [ ]:
show_doc(Wrangler.get_command_status)

<h4 id="Wrangler.get_command_status" class="doc_header"><code>Wrangler.get_command_status</code><a href="__main__.py#L36" class="source_link" style="float:right">[source]</a></h4>

> <code>Wrangler.get_command_status</code>(**`instance_id`**:`str`, **`command_id`**:`str`)



In [ ]:
wrangler = Wrangler()

In [ ]:
# Dictionary returned by execute_command
command_object = {
    'instance_id': 'i-0a0d845c32b78df5b',
    'command_id': '109f2cb8-7571-4c67-87bd-1e8a0f76ddba'
}

In [ ]:
#hide
# Mock the boto3 client's get_command_invocation so we can test the wrapper,
# not the core library.
wrangler._ssm_client.get_command_invocation.return_value = {"Status": "Success"}

In [ ]:
wrangler.get_command_status(**command_object)

'Success'

In [ ]:
#hide
assert wrangler.get_command_status(**command_object) == 'Success'

In [ ]:
show_doc(Wrangler.get_commands)

<h4 id="Wrangler.get_commands" class="doc_header"><code>Wrangler.get_commands</code><a href="__main__.py#L30" class="source_link" style="float:right">[source]</a></h4>

> <code>Wrangler.get_commands</code>(**`instance_id`**:`str`=*`None`*)



Optionally returns all commands invoked across all instances in region if no argument is passed. 

Otherwise, if an instance_id is given, returns all commands for that instance.

In [ ]:
wrangler = Wrangler()

In [ ]:
#hide
# Mock boto3 client object's list_commands method for no arguments
import datetime
from dateutil.tz import tzlocal
wrangler._ssm_client.list_commands.return_value = {'Commands': [{'CommandId': '109f2cb8-7571-4c67-87bd-1e8a0f76ddba',
   'DocumentName': 'AWS-RunShellScript',
   'DocumentVersion': '',
   'Comment': '',
   'ExpiresAfter': datetime.datetime(2020, 3, 24, 19, 9, 25, 841000, tzinfo=tzlocal()),
   'Parameters': {'commands': ['pwd']},
   'InstanceIds': ['i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 24, 17, 9, 25, 841000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '0',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': 'e4517c3c-15a9-4597-97a9-020e424e26bf',
   'DocumentName': 'AWS-RunShellScript',
   'DocumentVersion': '',
   'Comment': '',
   'ExpiresAfter': datetime.datetime(2020, 3, 24, 19, 6, 0, 955000, tzinfo=tzlocal()),
   'Parameters': {'commands': ['ls', 'pwd']},
   'InstanceIds': ['i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 24, 17, 6, 0, 955000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '0',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': '3b7d4252-9a97-4d1b-9d19-9c08744a8d59',
   'DocumentName': 'AWS-RunShellScript',
   'DocumentVersion': '',
   'Comment': '',
   'ExpiresAfter': datetime.datetime(2020, 3, 24, 18, 59, 17, 798000, tzinfo=tzlocal()),
   'Parameters': {'commands': ['pwd']},
   'InstanceIds': ['i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 24, 16, 59, 17, 798000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '0',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': 'ba6d0d3c-9b70-4bdd-ae0e-f9f8e5d4ff1b',
   'DocumentName': 'AWS-RunShellScript',
   'DocumentVersion': '',
   'Comment': '',
   'ExpiresAfter': datetime.datetime(2020, 3, 24, 18, 43, 17, 692000, tzinfo=tzlocal()),
   'Parameters': {'commands': ['ls', 'pwd']},
   'InstanceIds': ['i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 24, 16, 43, 17, 692000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '0',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': '829cc64d-d513-4e96-99f9-7b5b08fed824',
   'DocumentName': 'AWS-RunPatchBaseline',
   'DocumentVersion': '',
   'Comment': '7e7d1b69-5d9e-4c06-b14a-0a1b621988eb:b31ca646-213b-4226-bdc9-d01088a46fdb',
   'ExpiresAfter': datetime.datetime(2020, 3, 24, 13, 25, 56, 75000, tzinfo=tzlocal()),
   'Parameters': {'Operation': ['Scan']},
   'InstanceIds': ['i-072a37d0e881c5c54', 'i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 24, 12, 15, 56, 75000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '100%',
   'TargetCount': 2,
   'CompletedCount': 2,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': '8e6f457c-1e46-403f-b10a-775871f9785b',
   'DocumentName': 'AWS-RunShellScript',
   'DocumentVersion': '',
   'Comment': '',
   'ExpiresAfter': datetime.datetime(2020, 3, 23, 20, 42, 31, 747000, tzinfo=tzlocal()),
   'Parameters': {'commands': ['ls']},
   'InstanceIds': ['i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 23, 18, 42, 31, 747000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '0',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': '4b300847-21e5-4eaf-b7f7-76185418737f',
   'DocumentName': 'AWS-RunShellScript',
   'DocumentVersion': '',
   'Comment': '',
   'ExpiresAfter': datetime.datetime(2020, 3, 23, 15, 44, 37, 326000, tzinfo=tzlocal()),
   'Parameters': {'commands': ['pwd']},
   'InstanceIds': ['i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 23, 13, 44, 37, 326000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '0',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': 'ffe83ba7-d1bf-42e7-b9c2-e8fb5fc34b1b',
   'DocumentName': 'AWS-RunPatchBaseline',
   'DocumentVersion': '',
   'Comment': '7e7d1b69-5d9e-4c06-b14a-0a1b621988eb:4f17c2c7-c718-40be-9351-bf0ea5f915ca',
   'ExpiresAfter': datetime.datetime(2020, 3, 23, 14, 52, 9, 210000, tzinfo=tzlocal()),
   'Parameters': {'Operation': ['Scan']},
   'InstanceIds': ['i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 23, 13, 42, 9, 210000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '100%',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': '3eddfd83-6dd0-485b-b654-87f5b893775a',
   'DocumentName': 'AWS-UpdateSSMAgent',
   'DocumentVersion': '',
   'Comment': '46ca7721-6b10-4d65-bacd-9da0b8bd3c01:db2340ad-d99f-4dd8-bc94-478c46b96b28',
   'ExpiresAfter': datetime.datetime(2020, 3, 23, 14, 52, 9, 201000, tzinfo=tzlocal()),
   'Parameters': {},
   'InstanceIds': ['i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 23, 13, 42, 9, 201000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '100%',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': 'b3d9a63f-de75-4ba6-9812-9b1255e8d60e',
   'DocumentName': 'AWS-RunPatchBaseline',
   'DocumentVersion': '',
   'Comment': '7e7d1b69-5d9e-4c06-b14a-0a1b621988eb:4f17c2c7-c718-40be-9351-bf0ea5f915ca',
   'ExpiresAfter': datetime.datetime(2020, 3, 23, 14, 51, 36, 342000, tzinfo=tzlocal()),
   'Parameters': {'Operation': ['Scan']},
   'InstanceIds': ['i-072a37d0e881c5c54'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 23, 13, 41, 36, 342000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '100%',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': 'f038698d-6dfb-4b22-a7e3-2e6586fb33a2',
   'DocumentName': 'AWS-RefreshAssociation',
   'DocumentVersion': '',
   'Comment': 'Apply association with id at update time: afbd3596-7fd4-4281-baa3-af545c58e43c',
   'ExpiresAfter': datetime.datetime(2020, 3, 23, 14, 51, 35, 802000, tzinfo=tzlocal()),
   'Parameters': {'associationIds': ['afbd3596-7fd4-4281-baa3-af545c58e43c']},
   'InstanceIds': ['*'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 23, 13, 41, 35, 802000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '5',
   'MaxErrors': '0',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': '5bab1f58-600a-46ac-8c5d-57cbd9a2d7f9',
   'DocumentName': 'AWS-RunPatchBaseline',
   'DocumentVersion': '',
   'Comment': '7e7d1b69-5d9e-4c06-b14a-0a1b621988eb:07ed3a56-043c-4bc2-901b-88cd5639001d',
   'ExpiresAfter': datetime.datetime(2020, 3, 23, 14, 42, 39, 955000, tzinfo=tzlocal()),
   'Parameters': {'Operation': ['Scan']},
   'InstanceIds': ['i-072a37d0e881c5c54'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 23, 13, 32, 39, 955000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '100%',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': '85cee6b3-7700-4af0-bc9d-f53da438c4ca',
   'DocumentName': 'AWS-UpdateSSMAgent',
   'DocumentVersion': '',
   'Comment': '46ca7721-6b10-4d65-bacd-9da0b8bd3c01:db2340ad-d99f-4dd8-bc94-478c46b96b28',
   'ExpiresAfter': datetime.datetime(2020, 3, 23, 14, 42, 39, 941000, tzinfo=tzlocal()),
   'Parameters': {},
   'InstanceIds': ['i-072a37d0e881c5c54'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 23, 13, 32, 39, 941000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '100%',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': 'd370d759-151c-4955-ae5f-5ba467e40927',
   'DocumentName': 'AWS-RefreshAssociation',
   'DocumentVersion': '',
   'Comment': 'Apply association with id at update time: afbd3596-7fd4-4281-baa3-af545c58e43c',
   'ExpiresAfter': datetime.datetime(2020, 3, 23, 14, 42, 39, 535000, tzinfo=tzlocal()),
   'Parameters': {'associationIds': ['afbd3596-7fd4-4281-baa3-af545c58e43c']},
   'InstanceIds': ['*'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 23, 13, 32, 39, 535000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '5',
   'MaxErrors': '0',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': '97e70572-66c1-4213-a309-cfad977f7b6d',
   'DocumentName': 'AWS-UpdateSSMAgent',
   'DocumentVersion': '',
   'Comment': '46ca7721-6b10-4d65-bacd-9da0b8bd3c01:d07d1ee6-d2ce-4785-bf35-625757683b6d',
   'ExpiresAfter': datetime.datetime(2020, 3, 23, 14, 15, 8, 985000, tzinfo=tzlocal()),
   'Parameters': {},
   'InstanceIds': ['i-072a37d0e881c5c54'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 23, 13, 5, 8, 985000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '100%',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': 'c693967e-a77d-4b43-9d15-8435a1cfb685',
   'DocumentName': 'AWS-RefreshAssociation',
   'DocumentVersion': '',
   'Comment': 'Apply association with id at update time: afbd3596-7fd4-4281-baa3-af545c58e43c',
   'ExpiresAfter': datetime.datetime(2020, 3, 23, 14, 0, 28, 807000, tzinfo=tzlocal()),
   'Parameters': {'associationIds': ['afbd3596-7fd4-4281-baa3-af545c58e43c']},
   'InstanceIds': ['i-0631ecfea9497b859'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 23, 12, 50, 28, 807000, tzinfo=tzlocal()),
   'Status': 'TimedOut',
   'StatusDetails': 'DeliveryTimedOut',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '5',
   'MaxErrors': '0',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 1,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}}],
 'ResponseMetadata': {'RequestId': '1aa2a6d7-0aed-475c-9b0c-a059cc8ffd47',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1aa2a6d7-0aed-475c-9b0c-a059cc8ffd47',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '12023',
   'date': 'Tue, 24 Mar 2020 23:12:41 GMT'},
  'RetryAttempts': 0}}

In [ ]:
wrangler.get_commands()

{'Commands': [{'CommandId': '109f2cb8-7571-4c67-87bd-1e8a0f76ddba',
   'DocumentName': 'AWS-RunShellScript',
   'DocumentVersion': '',
   'Comment': '',
   'ExpiresAfter': datetime.datetime(2020, 3, 24, 19, 9, 25, 841000, tzinfo=tzlocal()),
   'Parameters': {'commands': ['pwd']},
   'InstanceIds': ['i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 24, 17, 9, 25, 841000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '0',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': 'e4517c3c-15a9-4597-97a9-020e424e26bf',
   'DocumentName': 'AWS

In [ ]:
#hide
# We don't care about the result, because this is a naked return
# of the boto3 client.  We just want to ensure that no args were
# passed to boto3 client for a naked call like this.
assert str(wrangler._ssm_client.list_commands.call_args) == "call()"

In [ ]:
#hide
# Mock the return value for cases where an instance is given
import datetime
from dateutil.tz import tzlocal
wrangler._ssm_client.list_commands.return_value = {'Commands': [{'CommandId': '829cc64d-d513-4e96-99f9-7b5b08fed824',
   'DocumentName': 'AWS-RunPatchBaseline',
   'DocumentVersion': '',
   'Comment': '7e7d1b69-5d9e-4c06-b14a-0a1b621988eb:b31ca646-213b-4226-bdc9-d01088a46fdb',
   'ExpiresAfter': datetime.datetime(2020, 3, 24, 13, 25, 56, 75000, tzinfo=tzlocal()),
   'Parameters': {'Operation': ['Scan']},
   'InstanceIds': ['i-072a37d0e881c5c54', 'i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 24, 12, 15, 56, 75000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '100%',
   'TargetCount': 2,
   'CompletedCount': 2,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': '8e6f457c-1e46-403f-b10a-775871f9785b',
   'DocumentName': 'AWS-RunShellScript',
   'DocumentVersion': '',
   'Comment': '',
   'ExpiresAfter': datetime.datetime(2020, 3, 23, 20, 42, 31, 747000, tzinfo=tzlocal()),
   'Parameters': {'commands': ['ls']},
   'InstanceIds': ['i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 23, 18, 42, 31, 747000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '0',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': '4b300847-21e5-4eaf-b7f7-76185418737f',
   'DocumentName': 'AWS-RunShellScript',
   'DocumentVersion': '',
   'Comment': '',
   'ExpiresAfter': datetime.datetime(2020, 3, 23, 15, 44, 37, 326000, tzinfo=tzlocal()),
   'Parameters': {'commands': ['pwd']},
   'InstanceIds': ['i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 23, 13, 44, 37, 326000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '0',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': 'ffe83ba7-d1bf-42e7-b9c2-e8fb5fc34b1b',
   'DocumentName': 'AWS-RunPatchBaseline',
   'DocumentVersion': '',
   'Comment': '7e7d1b69-5d9e-4c06-b14a-0a1b621988eb:4f17c2c7-c718-40be-9351-bf0ea5f915ca',
   'ExpiresAfter': datetime.datetime(2020, 3, 23, 14, 52, 9, 210000, tzinfo=tzlocal()),
   'Parameters': {'Operation': ['Scan']},
   'InstanceIds': ['i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 23, 13, 42, 9, 210000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '100%',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}},
  {'CommandId': '3eddfd83-6dd0-485b-b654-87f5b893775a',
   'DocumentName': 'AWS-UpdateSSMAgent',
   'DocumentVersion': '',
   'Comment': '46ca7721-6b10-4d65-bacd-9da0b8bd3c01:db2340ad-d99f-4dd8-bc94-478c46b96b28',
   'ExpiresAfter': datetime.datetime(2020, 3, 23, 14, 52, 9, 201000, tzinfo=tzlocal()),
   'Parameters': {},
   'InstanceIds': ['i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 23, 13, 42, 9, 201000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '100%',
   'TargetCount': 1,
   'CompletedCount': 1,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchOutputEnabled': False}}],
 'ResponseMetadata': {'RequestId': '6d6443a6-5d9c-4b02-a5e2-2bf721ed5e99',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6d6443a6-5d9c-4b02-a5e2-2bf721ed5e99',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '3736',
   'date': 'Tue, 24 Mar 2020 22:34:05 GMT'},
  'RetryAttempts': 0}}

In [ ]:
wrangler.get_commands('i-0a0d845c32b78df5b')

{'Commands': [{'CommandId': '829cc64d-d513-4e96-99f9-7b5b08fed824',
   'DocumentName': 'AWS-RunPatchBaseline',
   'DocumentVersion': '',
   'Comment': '7e7d1b69-5d9e-4c06-b14a-0a1b621988eb:b31ca646-213b-4226-bdc9-d01088a46fdb',
   'ExpiresAfter': datetime.datetime(2020, 3, 24, 13, 25, 56, 75000, tzinfo=tzlocal()),
   'Parameters': {'Operation': ['Scan']},
   'InstanceIds': ['i-072a37d0e881c5c54', 'i-0a0d845c32b78df5b'],
   'Targets': [],
   'RequestedDateTime': datetime.datetime(2020, 3, 24, 12, 15, 56, 75000, tzinfo=tzlocal()),
   'Status': 'Success',
   'StatusDetails': 'Success',
   'OutputS3BucketName': '',
   'OutputS3KeyPrefix': '',
   'MaxConcurrency': '50',
   'MaxErrors': '100%',
   'TargetCount': 2,
   'CompletedCount': 2,
   'ErrorCount': 0,
   'DeliveryTimedOutCount': 0,
   'ServiceRole': '',
   'NotificationConfig': {'NotificationArn': '',
    'NotificationEvents': [],
    'NotificationType': ''},
   'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
    'CloudWatchO

In [ ]:
#hide
assert str(wrangler._ssm_client.list_commands.call_args) == "call(InstanceId='i-0a0d845c32b78df5b')"